In [1]:
import numpy as np
from lxml import objectify

In [2]:
# angstroem, deg, kcal/mol, charge
# type, element, bond, angle, distance, energy, scale, charge
d_uff = np.dtype([
    ("type", np.unicode_, 8), 
    ("element", np.unicode_, 2), 
    ("bond", "d"),
    ("angle", "d"),
    ("distance", "d"),
    ("energy", "d"),
    ("scale", "d"),
    ("charge", "d"),
])
uff_data = np.loadtxt("uff.txt", dtype=d_uff)
#print(uff_data[0]["type"])

In [3]:
mass_dict = {}
with open("masses.txt") as f:
    for line in f.readlines():
        elem, mass = line.split()
        mass_dict[elem] = mass
#print(mass_dict)

In [4]:
# I don't want to have to redo all the smarts definitions
with open("uff-desc.xml", "r") as f:
    data=f.readlines()
xml_str = "".join(data)
root = objectify.fromstring(xml_str)

uff_dict = {}
for a in root.AtomTypes[0].Type:
    #print(a.attrib)
    try:
        defn = a.attrib['def']
    except KeyError:
        defn = None
    try:
        desc = a.attrib['desc']
    except KeyError:
        desc = None
    try:
        overrides = a.attrib['overrides']
    except KeyError:
        overrides = None
    a_dict = {
        "def" : defn,
        "desc" : desc,
        "overrides" : overrides
    }
    uff_dict[a.attrib["name"]] = a_dict

In [5]:
with open("uff-foyer.xml", "w") as f:
    f.write("<ForceField>\n\t<AtomTypes>\n")
    for atom in uff_data:
        try:
            mass = mass_dict[atom["element"]]
        except KeyError:
            continue
            
        a_str = ""
        for param in ["def", "overrides", "desc"]:
            try:
                val = uff_dict[atom["type"]][param]
            except KeyError:
                val = None
            if val != None:
                a_str += f' {param} = "{val}"'
                
        f.write(f'\t\t<Type name="{atom["type"]}" class="{atom["type"]}" element="{atom["element"]}" mass="{mass}"{a_str}/>\n')
    f.write('\t</AtomTypes>\n')
    f.write('\t<NonbondedForce coulomb14scale="0.833333" lj14scale="0.5">\n')
    for atom in uff_data:
        try:
            mass = mass_dict[atom["element"]]
        except KeyError:
            continue
        f.write(f'\t\t<Atom type="{atom["type"]}" charge="{atom["charge"]}" sigma="{atom["distance"]/10}" epsilon="{atom["energy"]}"/>\n')

    f.write('\t</NonbondedForce>\n')
    f.write('\t<HarmonicBondForce>\n')
    for i,atom1 in enumerate(uff_data):
        for atom2 in uff_data[i:]:
            # so in the rappe paper r_ij is supposed to be 
            # r_ij = r_i + r_j +r_BO + r_EN 
            # but to simplify things, I'm going to try just 
            # r_ij = r_i + r_j + 0.026
            # 0.026 is a correction so the c-c bond comes out right.
            length = atom1["bond"] + atom2["bond"] + 0.026 #angstroem
            length /= 10 # convert to nm
            
            # k_ij = 664.12 (Z*_i x Z*_j)/(r_ij^3)  unit is (kcal/mol)/A**3
            # unit for 664.12 must be A/(kcal/mol) -- divide by 10 to get nm
            energy = 4.184 * 664.12/10 * atom1["charge"] * atom2["charge"]/ length**3
            
            f.write(f'\t\t<Bond class1="{atom1["type"]}" class2="{atom2["type"]}" length="{length:.3f}" k="{energy:.3f}"/>\n')

    f.write('\t</HarmonicBondForce>\n')
    f.write('</ForceField>')